## **Classez des images**

### partie 2/4 : modèle perso

<br>

> #### notebook de mise en oeuvre de création et d’entraînement du modèle personnel, des simulations des différentes valeurs des hyperparamètres et de data augmentation.. <br><br>

<br>


## 0 Imports


### 0.1 Librairies, réglages


In [ ]:
# paths, folders/files
import os, sys, random, re
from os import listdir
from glob import glob
from zipfile import ZipFile
import time

# math, dataframes
import numpy as np
import pandas as pd
from pandarallel import pandarallel
from collections import Counter

# Visualisation
from pprint import pprint
import matplotlib.pyplot as plt
from matplotlib.image import imread
import seaborn as sns
import plotly.express as px
# from wordcloud import WordCloud
# from PIL import Image

# Feature engineering
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn import manifold, decomposition
from sklearn import cluster, metrics
from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import CountVectorizer

# NN
import tensorflow as tf
from tensorflow.keras.metrics import Accuracy, Precision, Recall, AUC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalAveragePooling1D, Flatten, Dense, Dropout
from tensorflow.keras.layers import Rescaling, RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical


print('\nPython version ' + sys.version)
print('Tensorflow version ' + tf.__version__)
print('Keras version ' + tf.keras.__version__)

# plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)

# Modify if necessary
num_cores = os.cpu_count()
print(f"\nNumber of CPU cores: {num_cores}")
pandarallel.initialize(progress_bar=False, nb_workers=6)


### 0.2 Fonctions


In [ ]:
model_results = []

def affichage_results():
    """Tracking manuel de nos modèles (data, params, tps, scores) pour comparaison."""

    # Create a DataFrame from the list of model results
    model_comparison_df = pd.concat([pd.DataFrame(model_results)], ignore_index=True)

    # Sort the DataFrame by precision in descending order (higher is better)
    model_comparison_df.sort_values(by='accuracy_val_moy', ascending=False, inplace=True)

    # Display the sorted DataFrame
    display(model_comparison_df)


### 0.3 Variables globales


In [ ]:
nb_classes = 3      # min 2, max 120

size_wh = 128
target_size=(size_wh, size_wh) # pour grille 5x5, stride (2,2) ?

alea = 42 # pour fixer les ttsplits et tjs travailler sur les mm datasets
# En revanche l'initialisation des poids des modèles restera aléatoire,
# pour pouvoir comparer les resultats sur +ieurs runs.

epochs = 10


### 0.4 Metriques


In [ ]:
# J'utiliserai tjs les noms anglais des métriques ici,
# pour éviter la confusion précision (fr) != precision (en),
# et pour simplement garder les noms des fonctions importées depuis tf.keras.metrics

# Nous sommes dans un cas de classification "classique", 1 classe prédite.
# Une première métrique simple et intuitive est donc l'accuracy :
# nb de prédictions correctes / nb total de prédictions.
# Cette métrique nous suffit déjà pour comparer et optimiser nos modèles.

# Si l'on souhaite étudier + en détail les prédictions des modèles, on utilisera
# la precison et le recall

# Precison (TP / (TP + FP)) :
# Une précision élevée signifie que si une classe est prédite par le modèle,
# alors il y a une forte probabilité (égale à la precision)
# que le chien appartienne en effet à cette classe.

# Recall (TP / (TP + FN)) :
# pour évaluer la capacité des modèles à identifier toutes les instances positives.
# Exemple : Si notre precision est égale à 1, c'est parfais, cela signifie que
# pour une une classe au moins, toutes les valeurs prédites par le modèle sont correctes.
# Cependant, il est possible que cela ne concerne que très peu de cas (mettons, 1 ou 2 prédictions)
# et qu'à côté de cela le modèle a pu faire des milliers de prédictions incorrectes,
# la precision seule ne nous le dit pas.

# Comme l'amélioration de la precision se fait svt au détriment du recall, en pratique
# on combine souvent les 2 avec le f1score (= moyenne harmonique)
# (2 x precision x recall) / (precision + recall)
# qui nous donne directement une idée du compromis precision / recall

# petit souci : le f1score et keras, c'est tout une histoire...
# Dans les versions récentes (depuis la 2.15.0 il me semble), le f1score est directement intégré
# au module metrics. Le problème est que conda n'arrive pas à résoudre un env avec ces versions,
# incompatibles avec les requirements d'autres packages dans l'env.

# Dans les versions + anciennes de keras, le f1score était dans un autre module, "addons",
# mais ce moule est désormais déprécié.
# Solution : on va juste faire un f1score custom ?
# ??

def f1score(y_true, y_pred):
    precision = Precision(y_true, y_pred)
    recall = Recall(y_true, y_pred)
    f1 = (2*precision*recall) / (precision + recall)

    return f1


metrics=[
    'Accuracy',
    # f1score,
    # AUC(),
]


### 0.4 Data


In [ ]:
data = pd.read_csv('./data/data_3_classes.csv', sep=',')

print(data.shape)
data.head()


### 0.5 Etude de faisabilité (sort of)


In [ ]:
# Ici l'étude de faisabilité préconisée par la méthode Agile n'est pas vraiment utile en tant que telle
# (On sait que le projet est faisable). Il s'agit plutôt de pouvoir observer le travail effectué par le
# bloc d'encodage, sans utiliser d'algorithme de prediction supervisée
# (algo classique ou plutôt, ici, bloc des layers fully connected)

images_features = []

for image_file in data["photo_path"] :
    image = load_img(image_file, target_size=(180, 180))
    image = img_to_array(image)
    images_features.append(image)

images_features = np.asarray(images_features)
images_features.shape


In [ ]:
# Reshape images to flatten them into vectors
flattened_images = images_features.reshape(images_features.shape[0], -1)
print(flattened_images.shape)

# Normalize the data
scaler = StandardScaler()
normalized_images = scaler.fit_transform(flattened_images)


### 0.6 Réduction dim


In [ ]:
# PCA

print(normalized_images.shape) # same as flattened_images

pca = decomposition.PCA(n_components=0.99)
feat_pca= pca.fit_transform(normalized_images)

print(feat_pca.shape)

# dimention divisée par 200 (presque), en conservant 99% de la variance !


In [ ]:
# Plot explained variance ratio
plt.figure(figsize=(8, 6))
plt.plot(range(1, pca.n_components_ + 1), pca.explained_variance_ratio_.cumsum(), marker='o', linestyle='--', color='#3af')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance Ratio vs. Number of Principal Components')
plt.grid(True)
plt.show()

# Pourquoi on a besoin du tsne pour la visu : en 2D ou même en 3D, les 3 premiers vecteurs propres
# # fournis par la PCA ne captent "que" (environ) un tiers de l'information.
# Ce qu'on verrait serait très déformé par les projections successives de la PCA.
# tester ?


### 0.7 tsne


In [ ]:
# t-sne

tsne = manifold.TSNE(n_components=2, perplexity=30, n_iter=2000, init='random', random_state=6)
X_tsne = tsne.fit_transform(feat_pca)


In [ ]:
# encodage target

label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(data["breed"])

data["target"] = label_encoder.transform(data["breed"])

display(data.head(1))
data.tail(1)


In [ ]:
df_tsne = pd.DataFrame(X_tsne, columns=['tsne1', 'tsne2'])
df_tsne["class"] = data["target"]

plt.figure(figsize=(8,5))
sns.scatterplot(
    x="tsne1", y="tsne2",
    hue="class",
    palette=sns.color_palette('tab10', n_colors=3), s=50, alpha=0.6,
    data=df_tsne,
    legend="brief")

plt.title('TSNE selon les vraies classes', fontsize = 30, pad = 35, fontweight = 'bold')
plt.xlabel('tsne1', fontsize = 26, fontweight = 'bold')
plt.ylabel('tsne2', fontsize = 26, fontweight = 'bold')
plt.legend(prop={'size': 14})

plt.show()

# Ca marche moins bien sans extraction de features !
# On retentera en fin de notebook, en utilisant notre modèle.


### 1 Création d'un premier modèle


In [ ]:
# Notre objectifs principal ici est
# de pouvoir observer / comprendre la fonction des différentes layers utilisées.

# Pour cela, nous allons commencer par une architecture très simple :
# le but n'est pas d'avoir le modèle le + performant possible.
# (irréaliste ici car on n'aurait ni le tps ni les ressources pour l'entrainer)
# (en revanche, voir le notebook 3, transfer learning, pour une comparaison de modèles + complexes)

# Première idée :
# Notre modèle de base sera donc inspiré d'AlexNet, dont l'architecture est :

# "AlexNet contains eight layers: the first five are convolutional layers,
# some of them followed by max-pooling layers, and the last three are fully connected layers.
# [...] The entire structure can be written as:

# (CNN -> RN -> MP)^2 -> (CNN^3 -> MP) -> (FC -> DO)^2 -> Linear -> softmax

# where
# CNN = convolutional layer (with ReLU activation)
# RN = local response normalization
# MP = maxpooling
# FC = fully connected layer (with ReLU activation)
# Linear = fully connected layer (without activation)
# DO = dropout

# It used the non-saturating ReLU activation function, which showed improved training performance
# over tanh and sigmoid." (wiki)


### 1.1 LeNet inspired architecture


In [ ]:
# Problème : 8 groupes de layers... (16 individuelles, en fait) C'est déjà beaucoup !
# On peut faire + simple, au moins pour commencer.

# Voyons de quoi sera capable un modèle inspiré plutôt par LeNet-5
# et par ce notebook : https://www.kaggle.com/code/schmoyote/simple-cnn-architecture-for-image-classification/notebook

model = Sequential()
model.add(Conv2D(6, kernel_size=(5, 5), activation='tanh', input_shape=(size_wh, size_wh, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, kernel_size=(5, 5), activation='tanh'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
# model.add(Dense(120, activation='tanh'))
model.add(Dense(60, activation='tanh'))
model.add(Dense(nb_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)

model.summary()


### 1.2 feature engineering


In [ ]:
feature = "photo_path"

X_feature = []

for image_file in data[feature] :
    image = load_img(image_file, target_size=target_size)
    image = img_to_array(image)
    X_feature.append(image)

X_feature = np.asarray(X_feature)

print("Shape of X_train:", X_feature.shape)
# ok


### 1.3 label encoding target


In [ ]:
y_target = np.asarray(data["target"])
print(y_target.shape)
pprint(y_target)


In [ ]:
# delete ? move ?

# Ici une simple normalisation de la valeur des pixels
# + transfo en tensor pour tf

def load_and_normalize(img_address):
    image = load_img(img_address, target_size=(180, 180)) # redondant (target_size)
    image = img_to_array(image)
    # image = tf.convert_to_tensor(image)
    return image


# data['input'] = data['denoised'].apply(load_and_normalize)

# data.head(1)


### 1.4 train test split


In [ ]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X_feature, y_target, test_size=0.1,
                                                            shuffle=True, random_state=alea,
                                                            stratify=y_target) # important

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1,
                                                            shuffle=True, random_state=alea,
                                                            stratify=y_train_val)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape, '\n')

print(y_train.shape)
print(y_val.shape)
print(y_test.shape, '\n')


### 1.5 one hot encoding (targets)


In [ ]:
# One-hot encode target values after the split to avoid data leakage

y_train_ohe = tf.keras.utils.to_categorical(y_train)
y_val_ohe = tf.keras.utils.to_categorical(y_val)
y_test_ohe = tf.keras.utils.to_categorical(y_test)


### 1.6 Training


In [ ]:
# Train the model
model.fit(X_train, y_train_ohe, epochs=epochs, batch_size=32,
          validation_data=(X_val, y_val_ohe))


### 1.7 Evaluation


In [ ]:
# On overfit dès le début ??

# Evaluate the model
val_loss_ref, val_acc_ref = model.evaluate(X_val, y_val_ohe)
print('Val accuracy:', val_acc_ref)

# En prédisant au hasard on aurait une chance sur 3, autrement dit
# ce modèle fait des prédictions quasi-aléatoires.
# Pas terrible, mais c'est un début !

# avant, tester sur photos d'origine (pour évaluer l'utilité du prétraitement effectué)


In [ ]:
# petit tracking manuel
results = {'model': 'V1',
            'df': 'data_3_classes',
            'feature': feature,
            'accuracy_val_moy': val_acc_ref,
            'time_fit': 'to do',
            'time_predict':'to do'
            }

# Append a new row for this model
model_results.append(results)


### 1.8 test utilité prétraitements


In [ ]:
def test_feature(df=data, feature='photo_path', epochs=epochs):  # photos d'origine, jpg, redim (<=> 'resized')

    # feature, target
    X_feature = []

    for image_file in df[feature]:
        image = load_img(image_file, target_size=target_size)
        image = img_to_array(image)
        X_feature.append(image)

    X_feature = np.asarray(X_feature)

    y_target = np.asarray(data["target"])

    X_train_val, X_test, y_train_val, y_test = train_test_split(X_feature, y_target, test_size=0.1,
                                                                shuffle=True, random_state=alea,
                                                                stratify=y_target) # important

    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1,
                                                                shuffle=True, random_state=alea,
                                                                stratify=y_train_val)

    y_train_ohe = tf.keras.utils.to_categorical(y_train)
    y_val_ohe = tf.keras.utils.to_categorical(y_val)
    y_test_ohe = tf.keras.utils.to_categorical(y_test)

    # model
    model = Sequential()
    model.add(Conv2D(6, kernel_size=(5, 5), activation='tanh', input_shape=(size_wh, size_wh, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16, kernel_size=(5, 5), activation='tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    # model.add(Dense(120, activation='tanh'))
    model.add(Dense(60, activation='tanh'))
    model.add(Dense(nb_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)

    model.fit(X_train, y_train_ohe, epochs=epochs, batch_size=32,
            validation_data=(X_val, y_val_ohe))

    _, val_acc = model.evaluate(X_val, y_val_ohe)
    print(f'Val accuracy (feature={feature}): {val_acc}')

    results = {'model': 'V1',
            'df': 'data_3_classes',
            'feature': feature,
            'accuracy_val_moy': val_acc,
            'time_fit': 'to do',
            'time_predict':'to do'
            }

    # Append a new row for this model
    model_results.append(results)


test_feature(df=data, feature='resized')

# rappel
print(f'Test accuracy (photo_path original): {val_acc_ref}')


# 0.6 de precision sans pretraitement (parfois 0.3 ??), 0.3 avec.
# Notre prétraitement semble (très) contre-productif. Ajuster dim, filtres ? (trop flou ?)
# Comme prétraitement, les méthodes .preprocessing() de keras consistent svt seulement en
# redimensionnemt + normalisation.
# faire pareil ?


In [ ]:
#

features_to_test = ['expo', 'contraste', 'denoised']

for feature in features_to_test:
    print(feature)
    test_feature(df=data, feature=feature)


# results
# 'resized' 0.32 ???
# 'expo' 0.63
# 'contraste' 0.32


### Comparaison


In [ ]:
affichage_results()

# On y voit déjà (un peu) + clair :
# Chaque étape de notre prétraitement semble + ou - détériorer la qualité des prédictions.

# Encore que... ?
# ??? resized et photo_path devraient donner des résultats bien + proches, non ??
# sets identiques sauf dim, et redim lors de création de X_feature
# ... devraient être exactement identiques, et donc avoir des resultats proches

# En fait d'un run à l'autre, les prédictions varient énormément...
# difficile du coup d'évaluer l'impact de nos prétraitements.
# moyenne sur +ieurs runs ?

model_results = []

# Svt le modèle ne parvient pas à "train ses layers", et l'accuracy des prédictions
# sur le jeu de validation reste au niveau de prédictions random.


### Mean multiple runs


In [ ]:
# mlflow ?

nb_runs = 10


def test_feature_n_times(df=data, feature='photo_path', epochs=epochs, n=nb_runs):
    """

    """
    results_val_acc, results_time_fit,  results_time_predict = [], [], []

    for i in range(n):

        print(f'\nRun {i}\n')

        # feature, target
        X_feature = []

        for image_file in df[feature]:
            image = load_img(image_file, target_size=target_size)
            image = img_to_array(image)
            X_feature.append(image)

        X_feature = np.asarray(X_feature)

        y_target = np.asarray(data["target"])

        X_train_val, X_test, y_train_val, y_test = train_test_split(X_feature, y_target, test_size=0.1,
                                                                    shuffle=True, random_state=alea,
                                                                    stratify=y_target) # important

        X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1,
                                                                    shuffle=True, random_state=alea,
                                                                    stratify=y_train_val)

        y_train_ohe = tf.keras.utils.to_categorical(y_train)
        y_val_ohe = tf.keras.utils.to_categorical(y_val)
        y_test_ohe = tf.keras.utils.to_categorical(y_test)

        # model
        model = Sequential()
        model.add(Conv2D(6, kernel_size=(5, 5), activation='tanh', input_shape=(size_wh, size_wh, 3)))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(16, kernel_size=(5, 5), activation='tanh'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Flatten())
        # model.add(Dense(120, activation='tanh'))
        model.add(Dense(60, activation='tanh'))
        model.add(Dense(nb_classes, activation='softmax'))

        # Compile the model
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # fit model and time it
        time_fit_start = time.time()
        model.fit(X_train, y_train_ohe, epochs=epochs, batch_size=32,
                validation_data=(X_val, y_val_ohe))
        time_fit_end = time.time()
        time_fit = time_fit_start - time_fit_end

        # time predictions
        time_predict_start = time.time()
        _, val_acc = model.evaluate(X_val, y_val_ohe)
        time_predict_end = time.time()
        time_predict = time_predict_start - time_predict_end

        print(f'Test accuracy (feature={feature}): {val_acc}')

        results_val_acc.append(val_acc)
        results_time_fit.append(time_fit)
        results_time_predict.append(time_predict)

    # moyennes
    mean_val_acc = np.mean(results_val_acc)
    mean_time_fit = np.mean(results_time_fit)
    mean_time_predict = np.mean(results_time_predict)

    # écarts-types (utile ici pour le score, afin d'avoir une idée de la "régularité" des résultats)
    # les tps d'entrainement / prédiction st bcp + stables
    std_val_acc = np.std(results_val_acc)

    results = {'model': 'V1',
        'df': 'data_3_classes',
        'feature': feature,
        'accuracy_val_moy': mean_val_acc,
        'accuracy_val_std': std_val_acc,
        'time_fit_moy': mean_time_fit,
        'time_predict_moy': mean_time_predict,
        }

    # Append a new row for this model
    model_results.append(results)


test_feature_n_times()

#


In [ ]:
test_feature_n_times(feature='resized')


In [ ]:
test_feature_n_times(feature='expo')


In [ ]:
test_feature_n_times(feature='contraste')


In [ ]:
test_feature_n_times(feature='denoised')


In [ ]:
affichage_results()

# model_results = []

# log / picle model. size ?


### Ameliorations


In [ ]:
# 3 pistes d'améliorations possibles,

# retour au preprocessing
# data augmentation
# model


In [ ]:
# Activation Functions: Tanh activation functions are commonly used in LeNet, but you might
# experiment with other activation functions like ReLU (Rectified Linear Unit), which tend to perform well
#  in many scenarios.

# Optimizer: The Adam optimizer is a good choice, but you might also experiment with other optimizers
# such as RMSprop or SGD (Stochastic Gradient Descent) with momentum.

# Loss Function: Cross-entropy loss is appropriate for classification tasks with softmax activation
# in the output layer, so categorical_crossentropy is fine.

# Model Capacity: LeNet is a relatively shallow network compared to modern architectures.
# Depending on the complexity of your dataset, you might need to adjust the model's capacity
# by adding more convolutional layers or increasing the number of units in the fully connected layers.

# Regularization: You may consider adding regularization techniques such as dropout or weight decay
# to prevent overfitting, especially if you observe overfitting during training.

# smaller grids ?
# modify input shape ?

# inception module + global average pooling ?
